In [ ]:
import numpy as np
import scipy 
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import ZebraLib as zb
import pyromat as pm
import CoolProp.CoolProp as CP
def K(C):
    return C+273.15

def MPa(psi):
    return psi*0.00689476

def K(C):
    return C+273.15

def get_air_properties(temperature):
    '''
    Temperature in K
    '''
    fluid = "Air"
    conductivity = CP.PropsSI("CONDUCTIVITY", "T", temperature, "P", 101325, fluid)
    viscosity = CP.PropsSI("VISCOSITY", "T", temperature, "P", 101325, fluid)
    prandtl = CP.PropsSI("PRANDTL", "T", temperature, "P", 101325, fluid)
    return conductivity, viscosity, prandtl
### Dados do problema
#dados físicos do compressor
a, b = 0.07, 0.09   #elipse
H1, H2 = 0.08, 0.13 #alturas do óleos
H = H1 + H2

#ELIPSE
#A_elip = np.pi*b*a
#P_elip = 2*np.pi*np.sqrt((b**2+a**2)/2)

#CILINDRO
r = (b+a)/2
A_elip = np.pi*r**2    #área base
P_elip = 2*np.pi*r     #perimetro
Lc = (H+r)           #comprimento caract
A_total =  P_elip*(H) + 2*A_elip 

#dados do ar em volta do compressor
T_amb1, T_amb2 = 57, 63
T_ambt = 30

#Dados do compressor
T1, T2 = 62, 68
Tt, Tb = 70, None
T_amb = K(28)          #média do ambiente 
Tm = K((T1+T2+Tt)/3)   #média sup do compressor
Tf = (Tm + T_amb)/2 #temp de filme
### Cálculo da convecção natural
#prop ar a 47°C
k, v_ar, Pr = get_air_properties(Tf)

def grashof(Ts):
    g = 9.81
    beta = 1/T_amb
    return (g*beta*(Ts - T_amb)*Lc**3)/v_ar**2

Ra_L = grashof(Tm)*Pr
crit = (35*H)/grashof(Tm)**(1/4)

print(f'Número de grashof é {grashof(Tm):.2e} --- Eq 9-15')
print(f'Número de Rayleigh é {Ra_L:.2e} --- usar correlação 9-19')
print(f'Critério para aprox pra placa {2*r:.2f}>={crit:.2f} tab 9-1 --> pode aprox pra placa vertical')

Nu = 0.59*Ra_L**(1/4)
h = (Nu*k)/Lc
print(f'Logo o h é {h:.2f} W/m^2k')
#conveccão
Q_conv = A_total*h*(Tm - T_amb)

#radiação
sig = 5.67E-8
alpha, G, ep= 0.8, 0, 0.6
Q_rad = (alpha*G + ep*sig*(Tm**4 - T_amb**4))*A_total

Q_total = Q_conv+Q_rad
print(f'O calor por convecção soma {Q_conv:.2f}W')
print(f'O calor por radiação é {Q_rad:.2f}W')
print(f'O calor total trocado é {Q_total:.2f}W')
print(f'Contribuição convecção {(Q_conv/Q_total)*100:.2f}% e radiação {(Q_rad/Q_total)*100:.2f}%')
